<a href="https://colab.research.google.com/github/giorgiagandolfi/laboratory_of_bioinformatics1/blob/master/Copy_of_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Generate the Hidden Markov Model

## Download from PDB

Using the advanced search in PDB we have selected the strucutres according with tyhe following fileds:

1. Resolution less than or equal to **3.5** A;
2. Annotated with the PFAM identifier of the Kunitz-BPTI domain: **PF00014**;
3. Number of polymer residues less than **100**.

Download the file in .csv format displaying sequences.

In [0]:
from google.colab import files
uploaded = files.upload()

Saving search_minepdb.csv to search_minepdb.csv


We have kept only the sequences having between 50 and 70 residues, since BPTI_Kunitz domain is genraly 58 residues lenght. 

In [0]:
!sed 's/\"//g' search_minepdb.csv | awk -F ',' '{if($6>50 && $6<70) print $0}' > clean.txt

In [0]:
! head clean.txt

We have converted the .csv file into a fasta format, in order to later perform the multiple sequence alignment.

In [0]:
!awk -F',' '{print ">"$1":"$2"\n"$5}' clean.txt > clean.fasta

In [0]:
!head clean.fasta

## BLASTclust

In order to remove reedundancy and to focus only on a smaller set, we have performed _blastclust_ command provided by blast package. The followed options have been used:

1. **-S**: score coverage threshold;
2. **-L**: lenght coverage threshold (default = 0.9);
3. **-i**: FASTA input file (_clean.fasta_);
4. **-o**: output file for list of clusters.

In our case, we decided to assign 99 to S and 0.99 to L.

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!cat clean_blastclust.out

Then I decided to keep only the best representatives of each cluster by looking to the best resolution structure. 

In [0]:
from google.colab import files
uploaded = files.upload()

Saving clean_blastclust_bestrep.out to clean_blastclust_bestrep.out


In [0]:
!cat clean_blastclust_bestrep.out

## Run PDBe fold

Now we run the multiple sequence alignment using PDBe fold. We simply paste the list of PDB IDs (_clean_blastclust_bestrep.out_). By looking at the output of PDBe fold, we can extract some important information: for example, **2FMA** strcutre has high RMSD (2.3534) and low quality score (0.2098). Then we download the multiple sequence alignment fasta file. 

## HMMBUILD

Before generating the HMM with _hmmbuild_ command, we have to modify the MSA fasta file in order to keep only the header and the sequence and transform everything in uppercase. Then we remove also **2FMA** sequence. The file that will be given as input to _hmmbuild_ is the following.

In [0]:
from google.colab import files
uploaded = files.upload()


In [0]:
!head clean_msa_kunitz.fasta

Then generate the HMM running _hmmbuild_ command.

# 2. Generate the datasets for model evaluation

## Download positives set

The positive set is constituted by all proteins stored in **SwissProt** containing the PFAM identifier for BPTI_Kunitz domain. After downloading the file in fasta format, we modify the file in order to have in the header only the sequence ID with the following command:

In [0]:
!awk '{if(substr($0,1,1)==">") {split($0,a,"|";print ">"a[2]} else {print $0}}'

In [0]:
from google.colab import files
uploaded = files.upload()

Saving all_kunitz.fasta to all_kunitz.fasta


In [0]:
!head -28 all_kunitz.fasta

## Cleaning of positives set

Before evaluating the performance of the such generated HMM of BPTI-Kuntiz domain, we have to remove from the positive set the proteins used to generate the model. In this way, we remove biases that could overfit our results. We will use a python program. The program takes in input two files: a files of identifiers and a file formatted as database. First, we have to generate the database using the command _formatdb_ that takes in input the fasta file of the sequences used to generate the HMM. 

In [0]:
#!formatdb -i clean_msa_kunitz.fasta -p

Then we perform a BLAST search between the such created database and the fasta files with all positives retrieved from SwissProt. **-m** and **-e** options are set respectively equal to 8 and 0.001. 

In [0]:
#!blastpgp -i all_kunitz.fasta -d clean_msa_kunitz.fasta -m 8 -e 0.001 -o all_kunitz_againstdb.bl8

Then we generate a .txt file containing all sequences of the BLAST search having an e-value equal to 100. This file will contain redundant sequences that are going to be removed from positive set.

In [0]:
#!awk '{if ($3==100) print $1}' all_kuntiz_againstdb.bl8 |sort -u > redundant_set.txt

In [0]:
from google.colab import files
uploaded = files.upload()

Saving redundant_set.txt to redundant_set.txt


In [0]:
!cat redundant_set.txt

P00974
P12111
P31713
P81658


Using a python program, the sequences corresponding to the UniProt IDs in the redundant_set.txt are removed from the positive set. The program is collected in the following link:
[python script](https://drive.google.com/drive/u/0/folders/19QBXaOWrR_8rQfaTMdEX8lC4CyBjrnb1)


In [0]:
#!./extract_seq.py redundant_set.txt all_kunitz.fasta > nonred_kunitz.fasta

## Generate positive sets

Before splitting the entire positive set into two sets to perform 2-fold cross validation, the protein sequences are random shuffled. 

In [0]:
#!grep ">" nonred_kuntiz.fasta | sed 's/>//' | sort -R > random1.txt

Then thee positive set is divided into two set of 177 and 178 protein sequences.

In [0]:
#!head -n 178 random1.txt > set1_r1_id1.txt
#!tail -n +179 random1.txt > set1_r1_id2.txt

## Download negative set

The negative set is constitued by all proteins in SwissProt that do not contain BPTI/Kunitz domain.In order to perform 2-fold cross validation, the negative set, consituted by 561894 protein sequences, is divided into two sets each one of 280947 proteins.

In [0]:
#!head -n 2111251 all_notkunitz.fasta > all_notkunitz_firsthalf.fasta
#!tail -n +2111252 all_notkunitz.fasta > all_notkunitz_secondhalf.fasta

Then we modify the file in order to have in the header only the sequence ID with the following command:

In [0]:
#!awk '{if(substr($0,1,1)==">") {split($0,a,"|";print ">"a[2]} else {print $0}}'all_notkunitz_firsthalf.fasta > negative_set1.txt
#!awk '{if(substr($0,1,1)==">") {split($0,a,"|";print ">"a[2]} else {print $0}}'all_notkunitz_secondhalf.fasta > negative_set2.txt

# 3. Method testing

Let's test the HMM model by using the HMMER **hmmsearch** program. hmmsearch command searches profile against a sequence database. Various options can be used. In this case, the BPTI/Kunitz profile HMM previously generated is searched against the positive set 1 and negative set 1, then it is searched against positive set 2 and negative set 2.

In [0]:
#!hmmsearch -Z 1 --noali --max --tblout positive_set1.hits bpti-kunitz.hmm positive_set1.txt
#!hmmsearch -Z 1 --noali --max --tblout negative_set1.hits bpti-kunitz.hmm negative_set1.txt

The following options are used:

1. **-Z 1** : normalization of e-value
2. **--noali** : don't output alignments, so output is smaller;
3. **--max** : turns all heuristic filters off (less speed, more power);
4. **--tblout** : save parseable table of per-sequence hits.


The output files are now modfied in order to have three columns:

1. first column: protein ID;
2. second column: e-value for the best domain (corresponding to column 8 of hhmsearch output) or for the entire sequence (corresponding to column 5);
3. third column: class identifier either **0**, in the case of negative, or **1**, in the case of positive.


In [0]:
#the parsing of the file is done according to hash character (#)
#---------------------------------------------------------------------
#!grep -v '^#' negative_set1.hits | awk '{print $1,$8,0}' >negative_set1.out  -----> class = 0
#!grep -v '^#' positive_set1.hits | awk '{print $1,$8,1}' >positive_set1.out  -----> class = 1

For optimization and testing of the performance, it is important to check the _hmmsearch_ output and include those proteins that have no match with the HMM. In particulary, only 137710 proteins in negative set have a match with the model. For classification purposes, the remaining proteins must be added to the _hmmsearch_ output assigning to them an e-value greater or equal than the e-value threshold and the class value of **0**. 

In [0]:
#find the proteins that do not match with the model and store them in a new file with e-value of 10 and class 0
#!comm <(grep "^>" negative_set1.txt | sed 's/>//' | sort) <(awk '{print $1}' negative_set1.out | sort) | awk -F '\t' '{if ($1!="") print $1,10,0}' > restore_set1.out
#
#add them to the negative set
#!cat negative_set1.out restore_set1.out >ok_negative_set1.out

# 4. Measure of performance

Now all data are collected in files containing protein ID, e-value and protein class(0=negative, 1= positive). Let's apply the python script calculating the performance of the model ([performance script](https://drive.google.com/open?id=1vFynNqnL12o-xUfgZ_2Rd54Cum0hAoun)). The program gives the following outputs for different thresholds, from 1 to 1e-20:
1. threshold;
2. accurancy;
3. Matthew correlation coefficient;
4. confusion matrix;
5. false positive rate;
6. true positive rate.

The last two parameters are useful for the construction of ROC curve.

In [0]:
#!performance.py <(cat ok_negative_set1.out positive_set1.out)